In [13]:
import os
from datetime import datetime

import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.models import alexnet, resnet50
from torchvision.transforms import functional as F
from xml.etree import ElementTree as ET
import pandas as pd
from PIL import Image

# models from torchvision
import torchvision.models as models
from torchvision.ops import RoIPool


from torchmetrics import Accuracy
from torchinfo import summary

from torch.utils.tensorboard import SummaryWriter


# Get Fast RCNN from torchvision


In [12]:
base_model = models.resnet50(weights=True)

# Remove the final fully connected layer (fc) to use as a feature extractor
modules = list(base_model.children())[:-2]  # Remove the last fully connected layer and avgpool
base_model = torch.nn.Sequential(*modules)


m:\ML\ML_regs\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
num_classes = 21  # PASCAL VOC classes + background
classifier = torch.nn.Sequential(
    torch.nn.Linear(512 * 7 * 7, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, 4096),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(4096, num_classes + 4)  # class scores and bbox regressor outputs
)

# RoI pooling
roi_pool = RoIPool(output_size=(7, 7), spatial_scale=1.0)